# Practica 4 - Chromadb

Alumno: Hernan Matias Silva

In [ ]:
#pip install -U transformers==4.35.0 sentence-transformers==2.2.2 chromadb==0.4.6 accelerate

# Creación de un Entorno de Python

En este documento, vamos a explicar cómo configurar un nuevo entorno de Python utilizando tanto `venv` como `conda`.

## Usando venv

`venv` es una herramienta que viene preinstalada con Python (versión 3.3 o superior) y permite crear entornos virtuales de Python aislados.

### Pasos

1. **Crear un entorno virtual:** Abre la terminal y ejecuta el siguiente comando en el directorio donde desees crear el entorno virtual.

   ```bash
   python3 -m venv nombre_del_entorno


2. **Activar el entorno virtual:** Utiliza el siguiente comando para activar el entorno.


  Linux/Mac:

```bash
    source nombre_del_entorno/bin/activate
```


   Windows:

```bash
    .\nombre_del_entorno\Scripts\activate
```


3. **Instalar dependencias desde requirements.txt con pip:** Una vez activado el entorno, puedes instalar todas las dependencias necesarias desde un archivo `requirements.txt` utilizando:

   ```bash
   pip install -r requirements.txt

4. **Desactivar el entorno virtual:** Para salir del entorno virtual, simplemente ejecuta:

```bash
   deactivate

## Usando Conda
conda es un sistema de gestión de paquetes y de entorno que puede instalar paquetes de diferentes lenguajes.

### Pasos
1. **Instalar Conda:** Puedes descargar e instalar Conda desde [este enlace](https://www.anaconda.com/download).

2. **Crear un entorno Conda:** Para crear un nuevo entorno con Conda, abre la terminal y ejecuta:

```bash
    conda create --name nombre_del_entorno python=3.9 ipykernel
```

Para esta biblioteca, vamos a usar python 3.9 y para interactuar con Jupyter Notebooks vamos a usar "ipykernel" o "jupyter"

3. **Activar el entorno Conda:** Utiliza el siguiente comando para activar el entorno.

```bash
    conda activate nombre_del_entorno
```
4. **Instalar dependencias desde requirements.txt con pip:** Al igual que con venv, puedes instalar las dependencias necesarias desde un archivo requirements.txt en el entorno Conda activado utilizando:
```bash
    pip install -r requirements.txt
```

5. Desactivar el entorno Conda: Para salir del entorno Conda, simplemente ejecuta:

```bash
    conda deactivate
```

In [25]:
# 1️⃣ Importar librerías
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import os
import numpy as np
import uuid
import requests
import json

# Modelo de embeddings
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)

def get_embeddings(text):
    return model.encode(text).tolist()

# Inicializar ChromaDB con persistencia
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_or_create_collection(name="documentos_ia")

def cargar_documentos():
    docs_path = "docs/"
    documentos = []
    metadatos = []
    ids = []

    for filename in os.listdir(docs_path):
        if filename.endswith(".txt"):
            with open(os.path.join(docs_path, filename), 'r', encoding="utf-8") as f:
                text = f.read().strip()
                chunks = [text[i:i+500] for i in range(0, len(text), 500)]
                
                for i, chunk in enumerate(chunks):
                    documentos.append(chunk)
                    metadatos.append({"source": filename})
                    ids.append(str(uuid.uuid4()))  # ID único
    
    return documentos, metadatos, ids

def crear_base_datos():
    docs, metas, ids = cargar_documentos()
    embeddings = [get_embeddings(d) for d in docs]
    collection.add(documents=docs, embeddings=embeddings, metadatas=metas, ids=ids)
    print("✅ Base de datos creada con éxito.")

# --- CRUD FUNCIONES ---

def create_example(new_doc):
    """Añadir un documento nuevo"""
    embedding = get_embeddings(new_doc)
    doc_id = str(uuid.uuid4())
    metadata = {"source": "nuevo_documento.txt"}
    
    collection.add(
        documents=[new_doc],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[doc_id]
    )
    print(f"✅ Documento añadido con ID: {doc_id}")

def read_example(query):
    """Consultar documentos similares"""
    query_emb = get_embeddings(query)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=1
    )
    
    if results["documents"]:
        print(f"📄 Mejor coincidencia: {results['documents'][0][0]}")
        print(f"📁 Fuente: {results['metadatas'][0][0]['source']}")
        print(f"🆔 ID: {results['ids'][0][0]}")
    else:
        print("⚠️ No se encontraron resultados.")

def get_documents_by_id(doc_id):
    """Obtener un documento por ID"""
    results = collection.get(ids=[doc_id], include=["documents", "metadatas", "embeddings"])
    
    if results["documents"]:
        print(f"📄 Documento: {results['documents'][0]}")
        print(f"📁 Fuente: {results['metadatas'][0]['source']}")
    else:
        print("⚠️ Documento no encontrado.")

def update_example(old_id, new_text):
    """Actualizar un documento (eliminar y reinsertar)"""
    collection.delete(ids=[old_id])
    
    new_id = str(uuid.uuid4())
    embedding = get_embeddings(new_text)
    metadata = {"source": "documento_actualizado.txt"}
    
    collection.add(
        documents=[new_text],
        embeddings=[embedding],
        metadatas=[metadata],
        ids=[new_id]
    )
    print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")

def delete_example(doc_id):
    """Eliminar por ID"""
    collection.delete(ids=[doc_id])
    print(f"🗑️ Documento con ID '{doc_id}' eliminado.")

In [26]:
crear_base_datos()

✅ Base de datos creada con éxito.


In [15]:
get_embeddings("Reynaldo González es un ingeniero de software con 10 años de experiencia en desarrollo web y móvil.")

[-0.09879041463136673,
 0.011900096200406551,
 -0.07934798300266266,
 -0.07173146307468414,
 0.04849894717335701,
 -0.08828753232955933,
 0.04442284256219864,
 0.06878751516342163,
 -0.04471663013100624,
 -0.019074415788054466,
 0.009390958584845066,
 0.0556383840739727,
 0.02038569562137127,
 0.005857520736753941,
 0.0571175254881382,
 0.03409785404801369,
 -0.033253442496061325,
 0.008881214074790478,
 0.022362586110830307,
 -0.06751402467489243,
 0.11720024794340134,
 -0.029926331713795662,
 0.0018126838840544224,
 -0.00767976138740778,
 -0.014972866512835026,
 0.0011125649325549603,
 0.006905629299581051,
 -0.016392141580581665,
 -0.025348829105496407,
 -0.09709931910037994,
 0.0348370224237442,
 0.06626584380865097,
 0.07979563623666763,
 0.050580158829689026,
 -0.024885069578886032,
 -0.00418727146461606,
 0.021157681941986084,
 -0.05807731673121452,
 -0.0793749988079071,
 0.012050828896462917,
 -0.1423081010580063,
 0.02995726279914379,
 -0.0037484760396182537,
 -0.0742330551147

In [16]:
results = collection.get(include=["embeddings", "documents", "metadatas"])

In [17]:
for id, content, metadata, embedding in zip(results["ids"], results["documents"], results["metadatas"], results["embeddings"]):
    print(f"ID: {id}, Contenido: {content[:100]} ", f"Metadata: {metadata}", f"Embedding: {embedding[:5]}...")

ID: 1c52b4c4-c49f-479a-a28a-0fa4f4c0b8c8, Contenido: El sol se alzaba sobre las colinas de Buenos Aires, pintando el cielo con tonos naranja y rosa. El a  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.05201021 -0.04184367  0.03102785  0.03516811  0.00390171]...
ID: 7052b785-9349-48ba-901b-7792b64fa052, Contenido: ciosa continuaba, con el tráfico constante y el murmullo de las conversaciones. Los turistas se mara  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.06561406  0.0027415   0.01852138 -0.00384713 -0.08065528]...
ID: 303974be-5fb5-4590-8bc2-8e879f1c6e0a, Contenido:  los restaurantes servían deliciosas parrilladas.

Hacia el norte, el elegante barrio de Recoleta in  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.06329251 -0.057605    0.01019359  0.01427127 -0.11836304]...
ID: a9d153e1-4b79-4d80-86ac-d915d108f351, Contenido: y novelas exploran la realidad, la fantasía y la condición humana con una prosa exquisita.

El fútbo  Metadata: {'source': 'doc1.txt'} Embedding: [ 0.10750

In [18]:
while True:
    opcion = input("Elige una opción o escribe ayuda: ")
    
    if opcion=="ayuda":
        print("\n--- Menú CRUD ---")
        print("1. Crear documento")
        print("2. Consultar por tema")
        print("3. Actualizar documento (por ID)")
        print("4. Eliminar documento (por ID)")
        print("5. Buscar documento (por ID)")
        print("6. Salir")

    if opcion == "1":
        nuevo_doc = input("Ingresa el texto a añadir: ")
        create_example(nuevo_doc)
        
    elif opcion == "2":
        consulta = input("Consulta: ")
        read_example(consulta)
        
    elif opcion == "3":
        doc_id = input("ID del documento a actualizar: ")
        nuevo_texto = input("Nuevo contenido: ")
        update_example(doc_id, nuevo_texto)
        
    elif opcion == "4":
        doc_id = input("ID del documento a eliminar: ")
        delete_example(doc_id)  

    elif opcion == "5":
        doc_id = input("ID del documento a buscar: ")
        get_documents_by_id(doc_id)
        
    elif opcion == "6":
        print("👋 Saliendo del programa.")
        break

KeyboardInterrupt: Interrupted by user

**Ejercicio 1: Crear y Consultar**

Crea un documento nuevo con el texto: "La gravedad es una fuerza fundamental en el 
universo". 
Realiza una consulta para encontrar documentos similares a la frase: "Fuerzas naturales 
que gobiernan el cosmos". 
 
¿Qué tipo de operación CRUD usaste primero? 
¿Cuál es el ID del documento creado? (Usa collection.peek()). 
Explica por qué el documento aparece como coincidencia en la consulta

In [ ]:
create_example("La gravedad es una fuerza fundamental en el universo")   # Operacion CRUD Crear
print(collection.peek(1))
read_example("Fuerzas naturales que gobiernan el cosmos")


✅ Documento añadido con ID: a60e3925-5acd-4473-86f4-50a814ec4b02
{'ids': ['1c52b4c4-c49f-479a-a28a-0fa4f4c0b8c8'], 'embeddings': array([[ 5.20102084e-02, -4.18436676e-02,  3.10278516e-02,
         3.51681113e-02,  3.90171120e-03,  5.41202389e-02,
         3.18245664e-02,  5.15107578e-03,  2.34087487e-03,
         7.27802841e-03,  5.65437414e-02,  7.95233157e-03,
        -5.68898469e-02, -6.39899671e-02,  5.74723408e-02,
         9.92390700e-03, -2.79309861e-02, -4.04784642e-03,
         3.22158411e-02,  1.06545240e-02,  7.93725029e-02,
        -5.20882644e-02, -5.48983924e-02,  8.65603089e-02,
        -7.93030635e-02,  5.29104210e-02, -8.08421243e-03,
        -1.72776952e-02, -9.94461998e-02, -4.68677469e-02,
         2.52617970e-02,  4.81346548e-02,  1.33476272e-01,
        -2.66981870e-02, -1.96807683e-02, -1.42063890e-02,
         1.37005478e-01, -1.16264917e-01,  3.77870277e-02,
         1.63287893e-02, -8.25200826e-02,  6.59070862e-03,
         1.33747561e-02, -3.46148619e-03, -6.

collection.peek() no devuelve el ultimo documento generado, devuelve documentos segun el orden de Chromadb.
El documento creado no fue retornado por la consulta quizas porque no es el mas cercano al calcular la distancia vectorial.

**Ejercicio 2: Actualizar y Eliminar**

Crea un documento con el texto: "Redes neuronales imitan procesos cerebrales". 
Encuentra su ID, luego actualízalo por: "Deep Learning es un subcampo de IA con redes neuronales profundas". 
Elimina el documento original usando el mismo ID. 

¿Qué ocurre si intentas actualizar un documento eliminado? 
¿Cómo se garantiza la integridad de los datos en las operaciones?

In [ ]:
new_doc = "Redes neuronales imitan procesos cerebrales"
embedding = get_embeddings(new_doc)
doc_id = str(uuid.uuid4())
metadata = {"source": "nuevo_documento.txt"}
    
collection.add(
    documents=[new_doc],
    embeddings=[embedding],
    metadatas=[metadata],
    ids=[doc_id]
)

print(f"✅ Documento añadido con ID: {doc_id}")

update_example(doc_id, "Deep Learning es un subcampo de IA con redes neuronales profundas")

# Eliminar el documento original
delete_example(doc_id)

✅ Documento añadido con ID: 5bf766eb-9c38-42b4-8f67-0bb6a4e30a07
♻️ Documento actualizado. Nuevo ID: d60a0a29-2855-45d3-ad4e-06c8bd85d4af
🗑️ Documento con ID '5bf766eb-9c38-42b4-8f67-0bb6a4e30a07' eliminado.
♻️ Documento actualizado. Nuevo ID: d60a0a29-2855-45d3-ad4e-06c8bd85d4af
🗑️ Documento con ID '5bf766eb-9c38-42b4-8f67-0bb6a4e30a07' eliminado.


Al intentar eliminar un documento ya eliminado la operacion no tendra efecto.
La integirdad de los datos se garantiza mediante ID unico. 

**Ejercicio 3: Errores y Validación**

Intenta eliminar un documento con el ID "no_existe". 
Crea un nuevo documento sin usar get_embeddings() (ejemplo: enviar un array vacío). 

¿Qué errores ocurren en cada caso? 
¿Cómo prevenir estos problemas?

In [ ]:
delete_example("no_existe")  

🗑️ Documento con ID 'no_existe' eliminado.


In [ ]:
El eliminar el documento que no existe no genera error, pero tampoco se elimina nada.

In [ ]:
doc_id = str(uuid.uuid4())
metadata = {"source": "nuevo_documento.txt"}

collection.add(
    documents=[new_doc],
    embeddings=None,  
    metadatas=[metadata],
    ids=[doc_id]
)
print(f"✅ Documento añadido con ID: {doc_id}")

C:\Users\herna\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:07<00:00, 10.9MiB/s]



✅ Documento añadido con ID: 1e249452-4874-4e3d-b6ed-651ba03e206a


Al crear un documento sin embedding se añade el documento. Pero puede generar problemas si se necesita el embeddings para las consultas.

**Ejercicio 4: Consultas por Tema**

Crea tres documentos: 
"Python es ideal para ciencia de datos" 
"JavaScript maneja interacciones web dinámicas" 
"Java se usa en aplicaciones empresariales" 
Realiza una consulta por la frase: "Lenguajes backend y frontend". 

¿Cuáles documentos aparecen como coincidencias? 
Analiza cómo las palabras clave influyen en los resultados.

In [ ]:
create_example("Python es ideal para ciencia de datos")
create_example("JavaScript maneja interacciones web dinámicas")
create_example("Java se usa en aplicaciones empresariales")
read_example("Lenguajes backend y frontend")

✅ Documento añadido con ID: 4201dd75-7055-48c5-8ca0-2de08824950f
✅ Documento añadido con ID: b9c7bfcd-fcd3-4e21-b67a-3d487ceb56d6
✅ Documento añadido con ID: b286b5f6-f3ba-4bcc-a22a-e0c0f5518a99
📄 Mejor coincidencia: lenguaje natural (PNL) es un campo de la inteligencia artificial que se centra en la comprensión y generación del lenguaje humano. Los embeddings son una herramienta fundamental en muchas tareas de PNL.

Los sistemas de recomendación utilizan bases de datos vectoriales para encontrar elementos similares a los que un usuario ha mostrado interés previamente. Esto permite ofrecer sugerencias personalizadas de productos, películas o música.

La detección de anomalías se basa en la identificación de 
📁 Fuente: doc1.txt
🆔 ID: 23ff0c99-6e31-4634-82bf-b1eb2774d7e2
✅ Documento añadido con ID: b9c7bfcd-fcd3-4e21-b67a-3d487ceb56d6
✅ Documento añadido con ID: b286b5f6-f3ba-4bcc-a22a-e0c0f5518a99
📄 Mejor coincidencia: lenguaje natural (PNL) es un campo de la inteligencia artificial que

Ningun documento de los creados aparece en las coincidencias.

**Ejercicio 5: Eliminaciones Selectivas**

Crea dos documentos con tópicos opuestos (ejemplo: uno sobre "Ecología" y otro sobre "Minería de datos"). 
Elimina el documento sobre ecología usando su ID. 
Verifica que solo quede un documento. 

¿Cómo usar collection.count() para confirmar?

In [ ]:
total_documentos = collection.count()
print(f"📊 Total de documentos en la colección: {total_documentos}")

new_doc = "Cuidar el medio ambiente no es una opción, es una responsabilidad."
embedding = get_embeddings(new_doc)
doc_id = str(uuid.uuid4())
metadata = {"source": "nuevo_documento.txt"}
    
collection.add(
    documents=[new_doc],
    embeddings=[embedding],
    metadatas=[metadata],
    ids=[doc_id]
)

print(f"✅ Documento añadido con ID: {doc_id}")

total_documentos = collection.count()
print(f"📊 Total de documentos en la colección luego de agregar ecolocia: {total_documentos}")

create_example("Cada dato cuenta una historia, la minería de datos nos ayuda a interpretarla.")

total_documentos = collection.count()
print(f"📊 Total de documentos en la colección luego de agregar mineria de datos: {total_documentos}")

delete_example(doc_id)

total_documentos = collection.count()
print(f"📊 Total de documentos en la colección luego de eliminar ecologia: {total_documentos}")


📊 Total de documentos en la colección: 148
✅ Documento añadido con ID: 680eb742-8564-4c82-92c5-8b8f4f6c36a8
📊 Total de documentos en la colección luego de agregar ecolocia: 149
✅ Documento añadido con ID: b019132e-9387-4c2a-a304-2e85008f1abd
📊 Total de documentos en la colección luego de agregar mineria de datos: 150
🗑️ Documento con ID '680eb742-8564-4c82-92c5-8b8f4f6c36a8' eliminado.
📊 Total de documentos en la colección luego de eliminar ecologia: 149


**Ejercicio 6: Actualización en Cadena**

Crea un documento con el texto "IA y ética son temas actuales". 
Actualízalo dos veces: 
Primera actualización: "Ética en IA aborda dilemas morales" 
Segunda actualización: "El uso de IA debe estar regulado por principios éticos". 

¿Cómo sigue evolucionando el contenido del documento con cada update?

In [ ]:
new_doc = "IA y ética son temas actuales"
embedding = get_embeddings(new_doc)
doc_id = str(uuid.uuid4())
metadata = {"source": "nuevo_documento.txt"}
    
collection.add(
    documents=[new_doc],
    embeddings=[embedding],
    metadatas=[metadata],
    ids=[doc_id]
)

print(f"✅ Documento añadido con ID: {doc_id}")

collection.delete(ids=[doc_id])

new_text = "Ética en IA aborda dilemas morales"
new_id = str(uuid.uuid4())
embedding = get_embeddings(new_text)
metadata = {"source": "documento_actualizado.txt"}

collection.add(
    documents=[new_text],
    embeddings=[embedding],
    metadatas=[metadata],
    ids=[new_id]
)
print(f"♻️ Documento actualizado. Nuevo ID: {new_id}")

update_example(new_id, "El uso de IA debe estar regulado por principios éticos")

✅ Documento añadido con ID: ea02faf8-d8f6-413f-9eaa-95b74ffd3fee
♻️ Documento actualizado. Nuevo ID: 9d8d2e3a-6759-416c-b64e-6da0f4c2fde7
♻️ Documento actualizado. Nuevo ID: b4a6952c-8c00-4edc-96cc-4d1ed047e3b0


El documento original es eliminado en cada modificacion.

**Ejercicio 7: Consultas Ambiguas**

Crea dos documentos similares pero no idénticos: 
"Blockchain garantiza transacciones seguras" 
"Criptomonedas usan tecnología blockchain". 
Realiza una consulta por "Seguridad en finanzas digitales". 

¿Ambos documentos aparecen? Explica la similitud semántica.

In [ ]:
create_example("Blockchain garantiza transacciones seguras")
create_example("Criptomonedas usan tecnología blockchain")
read_example("Seguridad en finanzas digitales")

✅ Documento añadido con ID: f5838e13-c0ac-4a85-8f21-aff09413b593
✅ Documento añadido con ID: f544fa68-0e76-45e9-ab2c-15e98dc8454d
📄 Mejor coincidencia: Blockchain garantiza transacciones seguras
📁 Fuente: nuevo_documento.txt
🆔 ID: f5838e13-c0ac-4a85-8f21-aff09413b593


Solo aparece el primer documento posiblemente por la palabra seguridad.

**Ejercicio 8: Reiniciar Base de Datos**

Elimina todos los documentos usando delete() con el parámetro correcto. 
Vuelve a cargar la base desde cero con crear_base_datos(). 

¿Cuántos documentos hay ahora? (Usa collection.count()).

In [28]:

total_documentos = collection.count()
print(f"📊 Total de documentos en la colección: {total_documentos}")

results = collection.get(where={"source": {"$exists": True}}, include=["ids"])
if results["ids"]:
    collection.delete(ids=results["ids"])
    print(f"🗑️ {len(results['ids'])} documentos eliminados.")
else:
    print("⚠️ No se encontraron documentos para eliminar.")

total_documentos = collection.count()
print(f"📊 Total de documentos en la colección luego de la eliminación: {total_documentos}")

crear_base_datos()

total_documentos = collection.count()
print(f"📊 Total de documentos en la colección luego de crear la base: {total_documentos}")

📊 Total de documentos en la colección: 219


ValueError: Expected where operator to be one of $gt, $gte, $lt, $lte, $ne, $eq, $in, $nin, got $exists in get.

**Ejercicio 9: Errores de Inserción**

Intenta crear un documento sin texto (ejemplo: cadena vacía). 
Corrige el error usando la función get_embeddings(). 

¿Qué lección aprendemos sobre los requisitos de ChromaDB?

In [19]:
create_example("")

✅ Documento añadido con ID: 2d4a22af-f427-409a-bf3d-bf3cf0abe0e0


**Ejercicio opcional: Retrieval-Augmented Generation**

Intente levantar un llm en un endpoint local con ollama o LMStudio. Utilice el llm para generar respuestas aumentadas con contexto similar a una consulta dada.

## ¿Cómo sigue esto?

### de forma conceptual debemos:
1. Recuperar el contexto relevante con una busqueda como read_example(query) o similar.
2. Formular el prompt para el llm, juntando la consulta original con los embeddings relevantes.
3. Enviar la Petición al Endpoint (en este caso LM Studio).
4. Recibir y mostrar la respuesta.

In [ ]:
# Esta sección es para la integración con LM Studio y no es parte del CRUD. Es solo un ejemplo de cómo se podría hacer una consulta a un modelo de lenguaje como Llama 3 o similar.
# 2️⃣ Integración con LM Studio (Llama 3 o similar)
import requests
import json

LM_STUDIO_URL = "http://192.168.0.3:1234/v1/chat/completions"  

def query_llama(prompt):
    """Envia un prompt a LM Studio y recibe la respuesta."""
    headers = {
        "Content-Type": "application/json"
    }
    data = {
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 256, 
        # Es posible agregar otros parámetros de generación aca (temperature, top_p, etc.)
    }
    try:
        response = requests.post(LM_STUDIO_URL, headers=headers, data=json.dumps(data))
        response.raise_for_status()
        return response.json()["choices"][0]["message"]["content"]
    except requests.exceptions.RequestException as e:
        print(f"Error al conectar con LM Studio: {e}")
        return None
    except (KeyError, json.JSONDecodeError) as e:
        print(f"Error al procesar la respuesta de LM Studio: {e}")
        return None

def responder_pregunta(pregunta):
    """Busca documentos relevantes y luego consulta a Llama."""
    query_emb = get_embeddings(pregunta)
    results = collection.query(
        query_embeddings=[query_emb],
        n_results=3  # Obtener algunos documentos relevantes como contexto
    )

    if results["documents"]:
        contexto = "\n\n".join(results["documents"][0])
        prompt = f"Basándote en la siguiente información:\n\n{contexto}\n\nResponde a la siguiente pregunta: {pregunta}" # Juntar el contexto obtenido y la pregunta
        respuesta = query_llama(prompt)
        return respuesta
    else:
        return "No se encontraron documentos relevantes para responder a tu pregunta."

# --- Ejemplo de uso ---
if __name__ == "__main__":
    crear_base_datos() 

    while True:
        pregunta_usuario = input("Pregúntame algo sobre los documentos (o escribe 'salir'): ")
        if pregunta_usuario.lower() == 'salir':
            break

        respuesta_ia = responder_pregunta(pregunta_usuario)
        if respuesta_ia:
            print(f"🤖 Respuesta de la IA: {respuesta_ia}")

✅ Base de datos creada con éxito.
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al conectar con Ollama: 404 Client Error: Not Found for url: http://localhost:11434/api/v1/generate
Error al con